### Restart and Run All Cells

In [2]:
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine, text

engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()

year = 2025
quarter = 1
current_time = datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time )

2025-07-26 11:15:02


In [3]:
cols = 'name year quarter q_amt_c q_amt_p inc_profit percent'.split()

format_dict = {
                'q_amt':'{:,}','q_amt_c':'{:,}','q_amt_p':'{:,}','inc_profit':'{:,}',
                'yoy_gain':'{:,}','acc_gain':'{:,}',   
                'latest_amt':'{:,}','previous_amt':'{:,}','inc_amt':'{:,}',
                'q_eps':'{:.4f}','y_eps':'{:.4f}','aq_eps':'{:.4f}','ay_eps':'{:.4f}',
                'percent':'{:.2f}%','inc_pct':'{:.2f}%'
              }

In [4]:
sql = """
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s) 
OR (year = %s-1 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC
"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2025 AND quarter <= 1) 
OR (year = 2025-1 AND quarter >= 1+1)
ORDER BY year DESC, quarter DESC



In [5]:
dfc = pd.read_sql(sql, conlt)
dfc["Counter"] = 1
dfc_grp = dfc.groupby(["name"], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp["Counter"] == 4]
dfc_grp.shape

(207, 5)

In [6]:
sql = """
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s-1) 
OR (year = %s-1 AND quarter >= %s) 
ORDER BY year DESC, quarter DESC"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2025 AND quarter <= 1-1) 
OR (year = 2025-1 AND quarter >= 1) 
ORDER BY year DESC, quarter DESC


In [7]:
dfp = pd.read_sql(sql, conlt)
dfp["Counter"] = 1
dfp_grp = dfp.groupby(["name"], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp["Counter"] == 4]
dfp_grp.head().style.format(format_dict)

,name,year,quarter,q_amt,Counter
0,3BBIF,8096,10,"5,279,119",4
1,ACE,8096,10,"838,717",4
2,ADVANC,8096,10,"35,075,357",4
3,AEONTS,8096,10,"2,860,344",4
4,AH,8096,10,"746,961",4


In [8]:
dfp.name.unique().shape

(224,)

In [9]:
dfm = pd.merge(dfc_grp, dfp_grp, on="name", suffixes=(["_c", "_p"]), how="inner")
dfm["inc_profit"] = dfm["q_amt_c"] - dfm["q_amt_p"]
dfm["percent"] = round(dfm["inc_profit"] / abs(dfm["q_amt_p"]) * 100, 2)
dfm["year"] = year
dfm["quarter"] = "Q" + str(quarter)
df_percent = dfm[cols]
df_percent.head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,3BBIF,2025,Q1,"5,497,047","5,279,119","217,928",4.13%
1,ACE,2025,Q1,"809,620","838,717","-29,097",-3.47%
2,ADVANC,2025,Q1,"37,207,829","35,075,357","2,132,472",6.08%
3,AEONTS,2025,Q1,"3,107,917","2,860,344","247,573",8.66%
4,AH,2025,Q1,"733,651","746,961","-13,310",-1.78%


In [10]:
# Create the SQL query with parameter binding
sql = text("DELETE FROM qt_profits WHERE year = :year AND quarter = :quarter")

# Execute the query with parameters
params = {'year': year, 'quarter': f'Q{quarter}'}
rp = conlt.execute(sql, params)

# Print the number of rows affected
print("Rows deleted:", rp.rowcount)

Rows deleted: 205


In [11]:
sql = "SELECT name, id FROM tickers"
tickers = pd.read_sql(sql, conlt)
tickers.sample(5)

,name,id
86,CSS,131
260,SAMTEL,418
80,CPNCG,122
18,AMC,22
14,ALT,16


In [12]:
df_ins = pd.merge(df_percent, tickers, on="name", how="inner")
df_ins.head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
0,3BBIF,2025,Q1,"5,497,047","5,279,119","217,928",4.13%,234
1,ACE,2025,Q1,"809,620","838,717","-29,097",-3.47%,698
2,ADVANC,2025,Q1,"37,207,829","35,075,357","2,132,472",6.08%,6
3,AEONTS,2025,Q1,"3,107,917","2,860,344","247,573",8.66%,7
4,AH,2025,Q1,"733,651","746,961","-13,310",-1.78%,9


In [13]:
# Convert DataFrame to list of records
rcds = df_ins.values.tolist()

# Define column names in the same order as values
columns = ['name', 'year', 'quarter', 'latest_amt', 'previous_amt', 'inc_amt', 'inc_pct', 'ticker_id']

# SQL insert statement with named parameters
sql = text("""
    INSERT INTO qt_profits 
    (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id)
    VALUES (:name, :year, :quarter, :latest_amt, :previous_amt, :inc_amt, :inc_pct, :ticker_id)
""")

try:
    # Execute inserts
    for rcd in rcds:
        # Convert list to dictionary
        params = dict(zip(columns, rcd))
        conlt.execute(sql, params)
except Exception as e:
    raise e

### End of loop

In [15]:
criteria_1 = df_ins.q_amt_c > 440_000
df_ins.loc[criteria_1, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,3BBIF,2025,Q1,"5,497,047","5,279,119","217,928",4.13%
1,ACE,2025,Q1,"809,620","838,717","-29,097",-3.47%
2,ADVANC,2025,Q1,"37,207,829","35,075,357","2,132,472",6.08%
3,AEONTS,2025,Q1,"3,107,917","2,860,344","247,573",8.66%
4,AH,2025,Q1,"733,651","746,961","-13,310",-1.78%


In [16]:
criteria_2 = df_ins.q_amt_p > 400_000
df_ins.loc[criteria_2, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,3BBIF,2025,Q1,"5,497,047","5,279,119","217,928",4.13%
1,ACE,2025,Q1,"809,620","838,717","-29,097",-3.47%
2,ADVANC,2025,Q1,"37,207,829","35,075,357","2,132,472",6.08%
3,AEONTS,2025,Q1,"3,107,917","2,860,344","247,573",8.66%
4,AH,2025,Q1,"733,651","746,961","-13,310",-1.78%


In [17]:
criteria_3 = df_ins.percent > 10.00
df_ins.loc[criteria_3, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
9,AMATA,2025,Q1,"2,848,499","2,482,899","365,600",14.72%
29,BEC,2025,Q1,"125,179","96,284","28,895",30.01%
32,BGRIM,2025,Q1,"1,832,154","1,556,871","275,283",17.68%
34,BJC,2025,Q1,"4,664,809","4,001,403","663,406",16.58%
41,CK,2025,Q1,"1,607,142","1,445,903","161,239",11.15%


In [18]:
df_ins_criteria = criteria_1 & criteria_2 & criteria_3
df_ins.loc[df_ins_criteria, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
9,AMATA,2025,Q1,"2,848,499","2,482,899","365,600",14.72%
32,BGRIM,2025,Q1,"1,832,154","1,556,871","275,283",17.68%
34,BJC,2025,Q1,"4,664,809","4,001,403","663,406",16.58%
41,CK,2025,Q1,"1,607,142","1,445,903","161,239",11.15%
42,CKP,2025,Q1,"1,875,975","1,344,536","531,439",39.53%


In [19]:
df_ins[df_ins_criteria].sort_values(by=["percent"], ascending=[False]).head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
70,GULF,2025,Q1,"36,054,658","18,170,334","17,884,324",98.43%,653
162,STA,2025,Q1,"2,688,767","1,670,375","1,018,392",60.97%,481
172,TFG,2025,Q1,"5,029,477","3,166,564","1,862,913",58.83%,513
59,EGATIF,2025,Q1,"1,121,526","729,221","392,305",53.80%,156
42,CKP,2025,Q1,"1,875,975","1,344,536","531,439",39.53%,107


In [20]:
df_ins[df_ins_criteria].sort_values(by=["name"], ascending=[True]).head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
9,AMATA,2025,Q1,"2,848,499","2,482,899","365,600",14.72%,21
32,BGRIM,2025,Q1,"1,832,154","1,556,871","275,283",17.68%,59
34,BJC,2025,Q1,"4,664,809","4,001,403","663,406",16.58%,65
41,CK,2025,Q1,"1,607,142","1,445,903","161,239",11.15%,106
42,CKP,2025,Q1,"1,875,975","1,344,536","531,439",39.53%,107


In [21]:
df_ins[df_ins_criteria].sort_values(by=["name"], ascending=[True]).head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
9,AMATA,2025,Q1,"2,848,499","2,482,899","365,600",14.72%,21
32,BGRIM,2025,Q1,"1,832,154","1,556,871","275,283",17.68%,59
34,BJC,2025,Q1,"4,664,809","4,001,403","663,406",16.58%,65
41,CK,2025,Q1,"1,607,142","1,445,903","161,239",11.15%,106
42,CKP,2025,Q1,"1,875,975","1,344,536","531,439",39.53%,107


In [22]:
conlt.commit()
conlt.close()

In [23]:
current_time = datetime.now()
formatted_time = current_time.strftime("%Y:%m:%d %H:%M:%S")
print(formatted_time)

2025:07:26 11:15:02
